# Subtractor
Quantum computers have the function of being able to perform the same calculations as conventional computers. Check the circuit for Subtractor.

## What we'll learn this time
1. Implementing Binary Subtractor Using Quantum Gates
2. Using quantum superposition, we can realize multiple Substractor in a single circuit

## Install Blueqat
Install Blueqat from pip.

In [1]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## Subtraction of binary numbers
Subtractor uses a CCX gate to perform minus sign or a plus sign and a CX gate to perform Subtraction. In this case, we will do a quantum circuit of binary Subtraction, a-b = cd. This time, we will implement four types of Subtraction according to the values of a and b. Each Subtraction is,

0-0 = 00 => 0000  
0-1 = 11 => 0111  
1-0 = 01 => 1001  
1-1 = 00 => 1100  

The first two qubits are the input values a and b, and the second two qubits are the output values c and d. I've built a separate circuit for the input of a and b and a circuit for the addition, and used it several times. For data input, use the X gate to flip 0 to 1.

The circuit part of the additon looks like this. * is a controlled bit.

```
a ---X---*---X---*------- a
b -------*-------|---*--- b
0 -------X-------|---|--- c
0 ---------------X---X--- d
```

In [5]:
#import tools
from blueqat import Circuit

#subtraction part
subtractor = Circuit().x[0].ccx[0,1,2].x[0].cx[0,3].cx[1,3]

In [6]:
#0-0
(Circuit() + subtractor).m[:].run(shots=100)

Counter({'0000': 100})

In [7]:
#0-1
(Circuit().x[1] + subtractor).m[:].run(shots=100)

Counter({'0111': 100})

In [8]:
#1-0
(Circuit().x[0] + subtractor).m[:].run(shots=100)

Counter({'1001': 100})

In [9]:
#1-1
(Circuit().x[0,1] + subtractor).m[:].run(shots=100)

Counter({'1100': 100})

As you can see, we were able to calculate.

## subtraction using superposition
Now, instead of filling in the data one by one with the X-gate, let's use the H-gate to do the subtraction.

In [10]:
#Use H gate as input instead of X gate
(Circuit().h[0,1] + subtractor).m[:].run(shots=100)

Counter({'1001': 18, '0000': 21, '1100': 36, '0111': 25})

Using the Hadamal Gate, I got 4 answers, about 1/4 each. If you create a universal subtraction circuit like this, you can perform calculations using the superposition state.

## subtraction using entanglement
Next, instead of the H-gate, we try to subtract a-b=0 using quantum entanglement.

In [11]:
#Make an entangled state of 00 and 11
(Circuit().h[0].cx[0,1] + subtractor).m[:].run(shots=100)

Counter({'1100': 51, '0000': 49})

Since the input values 00 and 11 are entangled, the addition of these two will come out about half by half.

--------

## Advanced: General subtraction
It implements the subtraction of common decimal numbers to each other.  
The subtraction circuit can be implemented by reversing the addition circuit.

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F489c8ea6-130c-d44b-4bae-4d86f64556c2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e67fbf71779fa7ebb443e93663c56dc4">

Consider the circuit from the right. Input $a, a+b$ to output $b$.  
It can be displayed in binary numbers such as $a = a_n ... a_0$, $b = b_n ... b_0$.  
(where n is based on the larger number.)

$c_i$ is a carry down.  

### Carry down

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F00361854-e782-0da0-3386-f59f668a1ada.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=58d1d41b0893b16bfb24bfd546328ff2" width="60%">

The first part of the circuit combines these to obtain the carry down.

### Overflow bit
The discrimination is given by $b_{n+1}$. This is called the overflow bit.  

When $a < a+b$, $b_{n+1}$ = 0  
When $a > a+b$, $b_{n+1}$ = 1.

If $a > a+b$, $2^{n+1} - b$ will be output.

### Implementation

General subtraction is:

1. Using the reverse and sum circuits calculates the carry bit.  
2. Check the overflow and store it in $b_{n+1}$.  
3. Using the carry circuit calculate the defference.

$b$ is output in $b_n ... b_0$, and $b_{n+1}$ is the overflow bit.  
We'll prepare it for implementation. This is the same as the addition circuit.

In [1]:
from blueqat import Circuit

def carry(i):
    return Circuit().ccx[i+1,i+2,i+3].cx[i+1,i+2].ccx[i,i+2,i+3]

def carry_reverse(i):
    return Circuit().ccx[i,i+2,i+3].cx[i+1,i+2].ccx[i+1,i+2,i+3]

def sum_reverse(a):
    return Circuit().cx[a,a+2].cx[a+1,a+2]

def tobinary(A):
    return bin(A)[2:]

def digits(a,b): 
     # change to the binary
     aa = tobinary(a)  
     bb = tobinary(b)  
     alen = len(aa)  
     blen = len(bb)  

     # get the value n
     maxlen = max(alen,blen) 
     if alen>blen: 
         bb = bb.zfill(alen) 
     elif blen>alen: 
         aa = aa.zfill(blen) 

     # mapping
     str = '' 
     for i in range(maxlen): 
         str += '0' + aa[maxlen-i-1] + bb[maxlen-i-1] 
     str += '0' 
     return str

def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

def todecimal(A):
    return int(str(A),2) 

def getb(result): 
     str = result[-1] 
     digi = int((len(result)-1)/3) 
     for i in range(digi): 
         str += result[-2-i*3] 
     return todecimal(str)

### General circuit

In [2]:
def minus(a,ab): 
     # swap a for ab
     c = ab
     ab = a
     a = c

     # Mapping a binary number to a circuit
     qubits = len(digits(a,ab)) 
     cir1 = toX(digits(a,ab)) 
     digi = int((len(digits(a,ab))-1)/3) 

     # first carry circuit and sum reverse circuit
     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += sum_reverse(i*3)
         cir4 += carry(i*3) 

     # last digit
     cir3 = sum_reverse((digi-1)*3) + Circuit(qubits).cx[-3,-2] 

     # Output the subtraction to bi
     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry_reverse((digi-1-i)*3)

     result = (cir1 + cir4 + cir3 + cir2).m[:].run(shots=1) 
     return getb(result.most_common()[0][0])

Let's actually calculate it.

In [3]:
minus(8,2)

6

In [4]:
minus(4,2)

2

In [5]:
minus(50,24)

26

We've calculated it. Incidentally, for $a > a + b$

In [6]:
minus(2,4)

14

This is also calculated.

### References
V. Vedral, A. Barenco, A. Ekert, "Quantum Networks for Elementary Arithmetic Operations",  
(Submitted on 16 Nov 1995)  
https://arxiv.org/pdf/quant-ph/9511018.pdf